<a href="https://colab.research.google.com/github/JacopoMangiavacchi/Swift-TensorFlow-Sample-Notebooks/blob/master/FizzBuzz_Swift_TensorFlow.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import Foundation
import TensorFlow

In [0]:
func bitEncode(_ n: UInt, bits: Int = 10) -> [Float] {
    var array = [Float](repeating: 0, count: bits)
    for i in 0..<bits {
        array[bits - 1 - i] = Float(n >> i & 1)
    }
    return array
}

func labelEncode(_ n: UInt) -> [Float] {
    if n > 0 {
        if n % 15 == 0 {
            return [1, 0, 0, 0]
        }
      
        if n % 3 == 0 {
            return [0, 1, 0, 0]
        }
      
        if n % 5 == 0 {
            return [0, 0, 1, 0]
        }
    }
  
    return [0, 0, 0, 1]
}

In [0]:
let start = 1
let end = 1024
var X = [[Float]]()
var Y = [[Float]]()

for i in start..<end {
    X.append(bitEncode(UInt(i)))
    Y.append(labelEncode(UInt(i)))
}

In [0]:
struct FizzBuzz: Layer {
    var l1: Dense<Float>
    var l2: Dense<Float>
    var l3: Dense<Float>


    init(bits: Int = 1) {
        l1 = Dense<Float>(inputSize: bits, outputSize: 64, activation: relu)
        l2 = Dense<Float>(inputSize: 64, outputSize: 128, activation: relu)
        l3 = Dense<Float>(inputSize: 128, outputSize: 4, activation: softmax) // {$0}
    }
    
    @differentiable(wrt: (self, input))
    func applied(to input: Tensor<Float>) -> Tensor<Float> {
        let output1 = l1.applied(to: input)
        let output2 = l2.applied(to: output1)
        return l3.applied(to: output2)

    }
}

In [5]:
let x = Tensor<Float>(shape: [Int32(end - start), 10], scalars: Array(X.joined()))
let y = Tensor<Float>(shape: [Int32(end - start), 4], scalars: Array(Y.joined()))

print(x.shape)
print(y.shape)

TensorShape(dimensions: [1023, 10])
TensorShape(dimensions: [1023, 4])


In [0]:
let batchSize: Int32 = 64
let trainingIterations = Int32(end - start) / batchSize

func minibatch<T>(_ x: Tensor<T>, batchIndex: Int32) -> Tensor<T> {
  let start = batchIndex * batchSize
  return x[start..<start+batchSize]
}

In [7]:
let optimizer = RMSProp<FizzBuzz, Float>(learningRate: 0.003)
var model = FizzBuzz(bits: 10)


for epoch in 1...1000 {
    var totalLoss: Float = 0

    for i in 0..<trainingIterations {
        let xBatch = minibatch(x, batchIndex: i)
        let yBatch = minibatch(y, batchIndex: i)

        let gradients = gradient(at: model) { m -> Tensor<Float> in
            let ŷ = m.applied(to: xBatch)
            let batchLoss = softmaxCrossEntropy(logits: ŷ, labels: yBatch)
            totalLoss += batchLoss.scalarized()
            return batchLoss
        }
        optimizer.update(&model.allDifferentiableVariables, along: gradients)
    }


    if epoch % 10 == 0 {
        print("Epoch \(epoch) loss: \(totalLoss)")
    }
}

Epoch 10 loss: 17.876196
Epoch 20 loss: 16.841173
Epoch 30 loss: 16.22872
Epoch 40 loss: 15.685812
Epoch 50 loss: 15.611365
Epoch 60 loss: 14.896134
Epoch 70 loss: 14.856857
Epoch 80 loss: 14.717216
Epoch 90 loss: 14.72665
Epoch 100 loss: 14.390274
Epoch 110 loss: 14.558286
Epoch 120 loss: 14.384992
Epoch 130 loss: 14.344002
Epoch 140 loss: 14.248563
Epoch 150 loss: 13.791179
Epoch 160 loss: 13.931348
Epoch 170 loss: 13.498377
Epoch 180 loss: 13.255869
Epoch 190 loss: 13.236813
Epoch 200 loss: 13.104104
Epoch 210 loss: 13.055023
Epoch 220 loss: 12.981417
Epoch 230 loss: 12.92092
Epoch 240 loss: 12.920042
Epoch 250 loss: 13.061244
Epoch 260 loss: 12.97039
Epoch 270 loss: 12.856938
Epoch 280 loss: 12.792654
Epoch 290 loss: 12.805379
Epoch 300 loss: 12.795936
Epoch 310 loss: 12.774382
Epoch 320 loss: 12.809796
Epoch 330 loss: 12.795239
Epoch 340 loss: 12.77373
Epoch 350 loss: 12.7742815
Epoch 360 loss: 12.776327
Epoch 370 loss: 12.810679
Epoch 380 loss: 12.777397
Epoch 390 loss: 12.806236

In [0]:
var X_test = [[Float]]()
var Y_true = [[Float]]()

for i in 0..<100 {
    X_test.append(bitEncode(UInt(i)))
    Y_true.append(labelEncode(UInt(i)))
}

let xTest = Tensor<Float>(shape: [100, 10], scalars: Array(X_test.joined()))
let yTest = model.applied(to: xTest)

In [10]:
let yMax = yTest.argmax(squeezingAxis: 1).array
let yArray = Array(yTest.array)
let description = ["FizzBuzz", "Fizz    ", "Buzz    ", "        "]

for i in 0..<100 {
    print(i, description[Int(yMax[i].description)!], Y_true[i], yArray[i])
}

0 Fizz     [0.0, 0.0, 0.0, 1.0] [0.00017480283, 0.99977857, 1.0735129e-05, 3.5888545e-05]
1          [0.0, 0.0, 0.0, 1.0] [1.640878e-15, 5.6808105e-09, 6.627498e-08, 0.9999999]
2          [0.0, 0.0, 0.0, 1.0] [2.0018784e-19, 1.0994125e-08, 8.401037e-11, 1.0]
3 Fizz     [0.0, 1.0, 0.0, 0.0] [2.0318408e-13, 0.9997317, 1.2177414e-14, 0.0002682725]
4          [0.0, 0.0, 0.0, 1.0] [5.585562e-21, 8.955586e-12, 1.926891e-09, 1.0]
5 Buzz     [0.0, 0.0, 1.0, 0.0] [1.5192329e-09, 2.1561364e-10, 0.99962723, 0.00037280255]
6 Fizz     [0.0, 1.0, 0.0, 0.0] [2.2565824e-15, 0.9998828, 2.6082473e-11, 0.00011719136]
7          [0.0, 0.0, 0.0, 1.0] [3.8478158e-11, 1.211138e-12, 9.624843e-14, 1.0]
8          [0.0, 0.0, 0.0, 1.0] [4.3298448e-35, 1.527414e-08, 1.6623667e-25, 1.0]
9 Fizz     [0.0, 1.0, 0.0, 0.0] [1.0172619e-10, 0.99998796, 7.6383274e-17, 1.2030224e-05]
10 Buzz     [0.0, 0.0, 1.0, 0.0] [3.914779e-17, 1.6406824e-07, 0.98674345, 0.013256441]
11          [0.0, 0.0, 0.0, 1.0] [1.0887178e-12, 7.49